In [46]:
import pdfplumber
import pandas as pd

pdf_path = "/Users/kaiyu/Downloads/y3s1 BT4241/issue1009.pdf"

data = []
columns = ["TW", "LW", "TI", "HP", "Title", "Artist", "Company", "Catalogue No"]

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        no_of_lines = text.count("\n") + 1
        if "ARIA Top 100 Singles" not in text:
            continue
        # get actual page dimensions
        page_width = page.width
        page_height = page.height
        # define column boundaries relative to width
        # (these ratios are tuned for ARIA’s standard column layout)
        # column_ratios = {
        #     "TW": (0.04, 0.06),
        #     "LW": (0.06, 0.08),
        #     "TI": (0.08, 0.10),
        #     "HP": (0.10, 0.12),
        #     "Title": (0.12, 0.50),
        #     "Artist": (0.50, 0.88),
        #     "Company": (0.88, 0.93),
        #     "Catalogue No": (0.93, 0.97),
        # }
        
        column_ratios = {
            "TW": (0.10, 0.12),
            "LW": (0.12, 0.14),
            "TI": (0.16, 0.18),
            "HP": (0.20, 0.24),
            "Title": (0.24, 0.44),
            "Artist": (0.44, 0.70),
            "Company": (0.75, 0.83),
            "Catalogue No": (0.83, 0.95),
        }

        # extract text from each column using relative coordinates
        cols_data = {}
        for col, (x0_ratio, x1_ratio) in column_ratios.items():
            x0 = x0_ratio * page_width
            x1 = x1_ratio * page_width
            col_region = page.crop((x0, 170, x1, page_height - 200))
            col_text = col_region.extract_text()
            print(col, col_text.splitlines())

        # combine columns into aligned rows
        num_rows = max(len(v) for v in cols_data.values())
        for i in range(num_rows):
            row = [cols_data[col][i] if i < len(cols_data[col]) else "" for col in columns]
            data.append(row)

# build DataFrame
df = pd.DataFrame(data, columns=columns)

# keep only rows that look valid (TW should be numeric)
df = df[df["TW"].str.match(r"^\d+$", na=False)].reset_index(drop=True)

print(df.head(10))

TW ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50']
LW ['1', '3', '4', '1', '1', '2', '1', '5', '8', '1', '6', '1', '9', '1', '7', '1', '3', '2', '2', '1', '2', '1', '2', '2', '3', '3', '2', '2', '2', '1', '2', '3', '5', '3', '3', '3', '3', '4', '3', '-', '3', '4', '-', '4', '4', '4', '4', '4', '5']
TI ['13', '5', '13', '9', '8', '5', '7', '12', '5', '5', '14', '8', '2', '11', '10', '10', '3', '7', '6', '5', '25', '15', '5', '8', '4', '3', '15', '13', '9', '11', '14', '22', '8', '16', '19', '16', '14', '2', '21', '1', '45', '40', '1', '44', '15', '28', '29', '8', '4']
HP ['1', '3', '3', '5', '6', '7', '8', '5', '3', '10', '3', '10', '9', '1', '6', '11', '18', '19', '20', '19', '1', '1', '24', '13', '26', '27', '4', '15', '29', '9', '16', '3', '34', '3'

ValueError: max() iterable argument is empty

In [21]:
print(page_width, page_height)

595 842
